In [1]:
import numpy as np
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ScipyParticle, AdvectionRK4, ErrorCode, Variable, ParcelsRandom, DiffusionUniformKh, AdvectionDiffusionM1, AdvectionDiffusionEM
from datetime import timedelta
from glob import glob
import math
import xarray as xr

In [2]:
# Particle set

class OilParticle(ScipyParticle):
    U10 = Variable('U10', dtype=np.float32, initial = 0)
    V10 = Variable('V10', dtype=np.float32, initial = 0)
    wind_speed = Variable('wind_speed', dtype=np.float32, initial = 0)

In [3]:
# Add MOI field set

data_path = '/storage/shared/oceanparcels/input_data/MOi/'
ufiles = sorted(glob(data_path+'psy4v3r1/psy4v3r1-daily_U_2019-06-*.nc'))
vfiles = [f.replace('_U_', '_V_') for f in ufiles]
wfiles = [f.replace('_U_', '_W_') for f in ufiles]
mesh_mask = data_path + 'domain_ORCA0083-N006/coordinates.nc'

filenames = {'U': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': ufiles},
             'V': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': vfiles},
             'W': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': wfiles}}
variables = {'U': 'vozocrtx', 'V': 'vomecrty', 'W': 'vovecrtz'}
dimensions = {'U': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
              'V': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
              'W': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'}}

fieldset_moi = FieldSet.from_nemo(filenames, variables, dimensions)

         It will be opened with no decoding. Filling values might be wrongly parsed.


In [4]:
# Add wind fieldset from netcdf file

wind_dataset = xr.open_dataset('era_5_wind_data_june_2019.nc')

filenames_t_wind = {'U': 'era_5_wind_data_june_2019.nc', 'V': 'era_5_wind_data_june_2019.nc'}
variables_t_wind = {'U': 'u10', 'V': 'v10'}
dimensions_t_wind = {'U': {'lat': 'latitude', 'lon': 'longitude', 'time': 'time'},
              'V': {'lat': 'latitude', 'lon': 'longitude', 'time': 'time'}}

fieldset_wind = FieldSet.from_netcdf(filenames_t_wind, variables_t_wind, dimensions_t_wind)

In [5]:
# Add the wind fieldset as a wind factor, with 3% of the wind velocity affecting advection

u_wind = Field('U', wind_dataset['u10']*0.03, grid=fieldset_wind.U.grid)
v_wind = Field('V', wind_dataset['v10']*0.03, grid=fieldset_wind.V.grid)

fieldset_windfactor = FieldSet(u_wind, v_wind)

u_wind_total = Field('U10', wind_dataset['u10'], grid=fieldset_wind.U.grid)
v_wind_total = Field('V10', wind_dataset['v10'], grid=fieldset_wind.V.grid)

In [6]:
# Make the summed field set from MOI data and wind factor data

summed_fieldset = FieldSet(U=fieldset_moi.U + fieldset_windfactor.U, V=fieldset_moi.V + fieldset_windfactor.V)

summed_fieldset.add_field(u_wind_total)
summed_fieldset.add_field(v_wind_total)

In [7]:
# Delete particles if out of bounds

def DeleteParticle(particle, fieldset, time):
    particle.delete()

In [8]:
# Loading the wind speed data test, needed later for different kernels

def LoadingWindSpeedTest(particle, fieldset, time):
    
    particle.U10 = fieldset.U10[particle]
    particle.V10 = fieldset.V10[particle]

In [9]:
number_particles = 5

lonp = 2 * np.ones(number_particles)
latp = 60 * np.ones(number_particles)
depp = np.zeros(number_particles)

In [10]:
# Load particle set with the summed fieldset

pset = ParticleSet(fieldset=summed_fieldset, pclass=OilParticle, lon=lonp, lat=latp, depth=depp)

In [11]:
kernels = pset.Kernel(AdvectionRK4) + pset.Kernel(LoadingWindSpeedTest)

output_file = pset.ParticleFile(name="test_wind_speed_summed_fieldset", outputdt=timedelta(minutes=10))

pset.execute(kernels, runtime=timedelta(minutes=30), dt=timedelta(seconds=30), recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle}, output_file=output_file)

output_file.close()

INFO: Temporary output files are stored in out-BYALKTCB.
INFO: You can use "parcels_convert_npydir_to_netcdf out-BYALKTCB" to convert these to a NetCDF file during the run.
100% (1800.0 of 1800.0) |################| Elapsed Time: 0:00:30 Time:  0:00:30


In [12]:
ds = xr.open_dataset('test_wind_speed_summed_fieldset.nc')

ds

<xarray.Dataset>
Dimensions:     (traj: 5, obs: 4)
Dimensions without coordinates: traj, obs
Data variables:
    trajectory  (traj, obs) float64 ...
    time        (traj, obs) datetime64[ns] ...
    lat         (traj, obs) float64 ...
    lon         (traj, obs) float64 ...
    z           (traj, obs) float64 ...
    U10         (traj, obs) float32 ...
    V10         (traj, obs) float32 ...
    wind_speed  (traj, obs) float32 ...
Attributes:
    feature_type:           trajectory
    Conventions:            CF-1.6/CF-1.7
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_version:        v2.3.0-81-g6696f217
    parcels_mesh:           spherical